In [31]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [32]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-04 23:51:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-04-04 23:51:47 (6.58 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [35]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [52]:
df_filtered = df.filter("total_votes>20")
df_filtered.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15806601|R1BTWIBLYYVOV7|B00I1LM5SK|     878438426|Seiko Men's SSC22...|         Watches|          5|           30|         30|   N|                Y|Extremely Satisfied!|A beautiful timep...| 2015-08-31|
|         US|   14870011| R6F9VY91ADPLA|B002ZLE41I|     118278818|Grand Seiko Wrist...|         Watches|          1|    

In [57]:
import pyspark.sql.functions as F

In [58]:
import math

In [59]:
df_filtered2 = df_filtered.withColumn("helpfulRate", F.col("helpful_votes")/F.col("total_votes"))

In [60]:
df_filtered2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|        helpfulRate|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-------------------+
|         US|   15806601|R1BTWIBLYYVOV7|B00I1LM5SK|     878438426|Seiko Men's SSC22...|         Watches|          5|           30|         30|   N|                Y|Extremely Satisfied!|A beautiful timep...| 2015-08-31|                1.0|
|         US|   14870011| R6F9VY91ADPLA|

In [61]:
df_filtered3 = df_filtered2.filter("helpfulRate>.5")

In [62]:
df_filtered3.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|       helpfulRate|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   15806601|R1BTWIBLYYVOV7|B00I1LM5SK|     878438426|Seiko Men's SSC22...|         Watches|          5|           30|         30|   N|                Y|Extremely Satisfied!|A beautiful timep...| 2015-08-31|               1.0|
|         US|   32751852|R3PXNV89DFIXKV|B00O

In [63]:
vine_filtered = df_filtered3.filter("vine == 'Y'")

In [64]:
vine_filtered.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|       helpfulRate|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   53096363|R1B7M0OP3UNP6O|B00V0G9OGE|     297682929|Armitron Sport Wo...|         Watches|          5|           49|         52|   Y|                N|Lots of features ...|My wife has been ...| 2015-08-27|0.9423076923076923|
|         US|   51394788|R2UUV4UGGYMQG8|B00M

In [66]:
nonVine_filtered = df_filtered3.filter("vine == 'N'")

In [67]:
nonVine_filtered.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|       helpfulRate|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   15806601|R1BTWIBLYYVOV7|B00I1LM5SK|     878438426|Seiko Men's SSC22...|         Watches|          5|           30|         30|   N|                Y|Extremely Satisfied!|A beautiful timep...| 2015-08-31|               1.0|
|         US|   32751852|R3PXNV89DFIXKV|B00O

In [68]:
df.count()

960872

In [69]:
df_filtered.count()

8902

In [70]:
df_filtered2.count()

8902

In [71]:
vine_filtered.count()

43

In [72]:
nonVine_filtered.count()

7720

In [73]:
df_filtered3.count()

7763

In [81]:
nonVine_filtered.filter("star_rating=5").count()

4018

In [82]:
vine_filtered.filter("star_rating=5").count()

14

In [83]:
df.filter("star_rating=5").count()

571627

In [84]:
df_filtered.filter("star_rating=5").count()

4210